In [1]:
import pandas as pd
import pyarrow
from google.cloud import storage
import urllib.request
import os

In [2]:
init_url = f'https://github.com/DataTalksClub/nyc-tlc-data/releases/download'
CREDENTIALS_FILE = './gcs.json'
BUCKET_NAME = 'nyc-tl-data-01'

In [ ]:
def fetch(dataset_url: str, year: str, color: str) -> pd.DataFrame:
    if color == 'fhv':
        dtypes = {'dispatching_base_num':'str', 'PUlocationID':'Int64', 'DOlocationID':'Int64', 'SR_Flag':'Int64'}
        date_cols = ['pickup_datetime', 'dropOff_datetime']
    elif color == 'green':
        dtypes = {'VendorID':'Int64', 'store_and_fwd_flag':'str', 'RatecodeID':'Int64', 'PULocationID':'Int64', 'DOLocationID':'Int64', \
                  'passenger_count':'Int64', 'ehail_fee':'Float64', 'payment_type':'Int64', 'trip_type':'Int64'}
        date_cols = ['lpep_pickup_datetime', 'lpep_dropoff_datetime']
    else:
        dtypes = {'VendorID':'Int64', 'store_and_fwd_flag':'str', 'RatecodeID':'Int64', 'PULocationID':'Int64', 'DOLocationID':'Int64', \
                  'passenger_count':'Int64', 'payment_type':'Int64'}
        date_cols = ['tpep_pickup_datetime', 'tpep_dropoff_datetime']
        
    df = pd.read_csv(dataset_url, compression='gzip', parse_dates=date_cols, dtype=dtypes)
    return df

In [4]:
def upload_to_gcs(object_name: str, local_file: os.path):
    client = storage.Client.from_service_account_json(CREDENTIALS_FILE)
    bucket = client.bucket(BUCKET_NAME)
    blob = bucket.blob(object_name)
    blob.upload_from_filename(local_file)

In [5]:
def web_to_gcs(color: str, year: str):
    for i in range(1,13):
        month = f'{i:02d}'
        filename = f'{color}_tripdata_{year}-{month}'
        dataset_url = f'{init_url}/{color}/{filename}.csv.gz'
        df = fetch(dataset_url, color)

        cwd = os.getcwd()
        file_path = os.path.join(cwd, f'{filename}.parquet')
        df.to_parquet(file_path, engine='pyarrow')

        upload_to_gcs(f'{color}/{filename}.parquet', file_path)
        print(f'GCS: {color}/{filename}.parquet')

In [6]:
# web_to_gcs('green', '2019')
# web_to_gcs('green', '2020')
web_to_gcs('yellow', '2019')
web_to_gcs('yellow', '2020')

/Users/sandra/Desktop/6_dbt_cloud/.venv/lib/python3.13/site-packages/google_crc32c/__init__.py:29: RuntimeWarning: As the c extension couldn't be imported, `google-crc32c` is using a pure python implementation that is significantly slower. If possible, please configure a c build environment and compile the extension
  warnings.warn(_SLOW_CRC32C_WARNING, RuntimeWarning)


GCS: yellow/yellow_tripdata_2019-01.parquet
GCS: yellow/yellow_tripdata_2019-02.parquet
GCS: yellow/yellow_tripdata_2019-03.parquet
GCS: yellow/yellow_tripdata_2019-04.parquet
GCS: yellow/yellow_tripdata_2019-05.parquet
GCS: yellow/yellow_tripdata_2019-06.parquet
GCS: yellow/yellow_tripdata_2019-07.parquet
GCS: yellow/yellow_tripdata_2019-08.parquet
GCS: yellow/yellow_tripdata_2019-09.parquet
GCS: yellow/yellow_tripdata_2019-10.parquet
GCS: yellow/yellow_tripdata_2019-11.parquet
GCS: yellow/yellow_tripdata_2019-12.parquet
GCS: yellow/yellow_tripdata_2020-01.parquet
GCS: yellow/yellow_tripdata_2020-02.parquet
GCS: yellow/yellow_tripdata_2020-03.parquet
GCS: yellow/yellow_tripdata_2020-04.parquet
GCS: yellow/yellow_tripdata_2020-05.parquet
GCS: yellow/yellow_tripdata_2020-06.parquet
GCS: yellow/yellow_tripdata_2020-07.parquet
GCS: yellow/yellow_tripdata_2020-08.parquet
GCS: yellow/yellow_tripdata_2020-09.parquet
GCS: yellow/yellow_tripdata_2020-10.parquet
GCS: yellow/yellow_tripdata_2020